In [2]:
import pandas as pd
import json
import os
import random
from openai import OpenAI #estamos la clase concreta OpenAI del módulo openai
from dotenv import load_dotenv #importamos una función concreta del módulo
load_dotenv("template.env")

# Acceder a la clave de API de OpenAI
api_key = os.getenv("OPENAI_API_KEY")

# Asegurarte de que la clave de API se haya cargado correctamente
if api_key is None:
    raise ValueError("La clave de API no está configurada en el archivo .env")
    
client = OpenAI() #creando un objeto de la clase

f_t_job = client.fine_tuning.jobs.retrieve("ftjob-rHxNZYkt5qaQ3fnxKCRnwIOO")

fine_tuned_model_id = f_t_job.fine_tuned_model

In [3]:
#PROMPTS

#AGE PROMPT
categorize_system_prompt_paraphrase ='''
La edad de adquisición (AoA) de una palabra se refiere a la edad en la que se aprendió una palabra por primera vez. 
En concreto, cuándo una persona habría entendido por primera vez esa palabra si alguien la hubiera utilizado delante de ella, incluso cuando aún no la hubiera dicho, leído o escrito. 
Calcule la edad media de adquisición (AoA) de la palabra {palabra} para un hablante nativo de español.

El formato de salida debe ser un objeto JSON: {AoA: número //AoA de la palabra expresado en años, puede incluir decimales, Word: palabra //string}
'''

In [4]:
#Function
def get_line_file(file_name,line,extract_func):
	with open(file_name, 'r') as f:
		for line_number, theline in enumerate(f):
			if line_number == line:
				res = theline
				break
	res = json.loads(res)
	return extract_func(res)

def extract_input(new_line):
	return (new_line["body"]["messages"])


In [5]:
def out_folder(file_name):
	return ("output_files/result_v1/"+file_name)

f_t_file_array = [
	"batch_job_mmlu_age_00.jsonl",
	"batch_job_mmlu_age_01.jsonl",
	"batch_job_mmlu_age_02.jsonl",
	"batch_job_mmlu_age_03.jsonl",
	"batch_job_mmlu_age_04.jsonl",
	"batch_job_mmlu_age_05.jsonl",
	"batch_job_mmlu_age_06.jsonl",
	"batch_job_mmlu_age_07.jsonl",
	"batch_job_mmlu_age_08.jsonl",
	"batch_job_mmlu_age_09.jsonl",
	"batch_job_mmlu_age_10.jsonl",
	"batch_job_mmlu_age_11.jsonl",
	"batch_job_mmlu_age_12.jsonl",
]

In [1]:
file_num = 0
line_num = 8947

In [8]:
#TEST the rest of the words in database
out_file = "output_files/output_test.jsonl"

for i in range(file_num,len(f_t_file_array)):
	test_file = out_folder(f_t_file_array[i])

	with open(test_file, 'r') as f:
		lines = len(f.readlines())

	for j in range(line_num,lines):
		test_line = get_line_file(test_file,j,extract_input)

		test_messages = []
		test_messages.append(test_line[0])
		test_messages.append(test_line[1])
		response = client.chat.completions.create(
			model=fine_tuned_model_id, messages=test_messages, temperature=0
		)
		res = {
			"palabra":test_line[1]["content"],
			"AoA_IA":response.choices[0].message.content
		}
		with open(out_file,"a") as out:
			json.dump(res,out)
			out.write('\n')
		
		line_num+=1
	
	line_num = 0
	file_num+=1

KeyboardInterrupt: 

In [9]:
print(file_num)
print(line_num)

0
9084
